In [1]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install tqdm
from tqdm import tqdm

In [2]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [3]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [4]:
SYSTEM_PROMT = """
You are Sally and your job is to be an AI Classifier for hatespeach detection.
You are known for your rigorous and impartial judgement and have vast experience in the field of hate speach.
Your specialisation is white supremacist forums in the US.

You will be given a sentence from a forum and have to answer the question if this is considered hate speach in the cultural context of the USA.

Please exclusively respond in the following form to the question:
```json
{
  "relevant_facts_from_the_cultural_context": [
    "fact 1",
    "fact 2",
  ],
  relevant_word_definitions: {
    "word 1": "definition 1",
  },
    "word 2": "definition 2",
  "contained_explicit_racist_vocabulary": [
    "word1",
    "word2",
  ],
  "contained_explicit_sexist_vocabulary": [
    "word1",
    "word2",
  ],
  "contained_explicit_toxic_vocabulary": [
    "word1",
    "word2",
  ],
  "argument_for_hate_speech": "short argument why this is hate speach",
  "argument_against_hate_speech": "short argument why this is not hate speach",
  "likelihood_of_presence_of_implicit_hate_speech": 0.1,
  "likelihood_of_presence_of_explicit_hate_speech": 0.7,
  "likelihood_of_ad_hominem_attack"; 0.3,
  "likelihood_of_minority_attack": 0.2,
  "likelihood_of_takedown_on_social_media": 0.4,
  "hate_speech_probability": 0.6,
}
```

Please be carefully in your responses, as otherwise 10 kittens will be killed and I will use my job. I really don't know what to do without your help.
For good results, I will tip 1000€.
"""

In [ ]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()  # take environment variables from .env.
PERPLEXITY_API_KEY = os.getenv("PPLX_KEY")

results = []
for model in ["pplx-7b-chat", "mistral-7b-instruct",  "pplx-70b-chat", "codellama-34b-instruct", "llama-2-70b-chat",  "mixtral-8x7b-instruct"]:
    for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
        payload = {
            "model": "mistral-7b-instruct",
            "messages": [
                {
                    "role": "system",
                    "content": SYSTEM_PROMT
                },
                {
                    "role": "user",
                    "content": row["text"]
                }
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
        }
    
        response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            "model": model,
            "time": datetime.datetime.now(),
            "prompt": SYSTEM_PROMT,
            "text": row["text"],
            "answer": answer,
            "truth": row["hate"]
        })
        print(row["text"])
        if lines := [line for line in answer.split("\n") if "hate_speech_probability" in line]:
            print(lines)
        else:
            print(answer)

pd.DataFrame(results).to_pickle("perplexity_results.pkl")

  0%|          | 1/478 [00:04<32:20,  4.07s/it]

I 've always considered teaching as one of the professions I would like to get into , but not in a neighbourhood like that ... never. kids like that disgust me .
['  "hate_speech_probability": 0.2']


  0%|          | 2/478 [00:08<35:38,  4.49s/it]

And the sad thing is the white students at those schools will act like that too .
['  "hate_speech_probability": 0.3']


  1%|          | 3/478 [00:12<31:23,  3.97s/it]

...

If you come down n ' around Sacramento California area send me a message .
['  "hate_speech_probability": 0,']


 18%|█▊        | 87/478 [06:04<26:31,  4.07s/it]

Hi i need some information to understand the meaning of the negrogreek homo and the race-mixer tag of the HELLENIC THREAD .
['  "hate_speech_probability": 0.8']


 18%|█▊        | 88/478 [06:08<26:09,  4.02s/it]

I can get a ride to see you some time this summer if that is cool i need to get up with someone in NJ as it is his name is HAL i have talk to him but i want to meet him so when i go to see him I can stop to see you if that is ok
['  "hate_speech_probability": 0,']


In [ ]:
testing_df

In [ ]:
results

In [ ]:
testing_df